# preparations

## imports

In [1]:
import os
import sys
import json
import itertools

import pyspark.sql.functions as F # for SQL queries
import pyspark.sql.types as T
# !yarn application -kill application_1690360394514_7696

## functions and variables

In [2]:
import pyspark
def getNaNs(data:pyspark.sql.DataFrame=None) -> None:
    features = data.columns
    n_observations = data.count()
    
    for feature in features:
        n_wht_spcs = 25 - len(feature)
        n_NaNs = data.filter((data[feature] == "") | data[feature].isNull() |F.isnan(data[feature])).count()
        
        n_wht_spcs_dgts = 25 - len(str(n_NaNs))
        
        print(
            f'Number of NaN values:| {feature}', ' '*n_wht_spcs, f' | {n_NaNs}', 
            ' '*n_wht_spcs_dgts, f' | {(n_NaNs/n_observations)*100:.2f}%')

## spark context creation

In [3]:

import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: Failed to initialize Spark session.
Traceback (most recent call last):
  File "<string>", line 41, in <module>
AttributeError: type object 'SparkSession' has no attribute '_create_shell_session'


SystemExit: 1

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

# data analysis

## data load

In [ ]:
# initialize `lab directory`
lab_dir = '/labs'

items_inf   = spark.read.csv(f'{lab_dir}/slaba03/laba03_items.csv', header=True, sep='\t')
test_smpl   = spark.read.csv(f'{lab_dir}/slaba03/laba03_test.csv', header=True)
train_smpl  = spark.read.csv(f'{lab_dir}/slaba03/laba03_train.csv', header=True)
vws_prgrmms = spark.read.csv(f'{lab_dir}/slaba03/laba03_views_programmes.csv', header=True)

print(f'number of train_samples: {train_smpl.count()}; test_samples: {test_smpl.count()}')

## eda

### laba03_items.csv

In [ ]:
train_ftrs = train_smpl.join(
    other=items_inf.select('item_id', 'content_type', 'title', 'year', 'genres'),
    on=['item_id'], how='left'
)
# getNaNs(data=train_ftrs)

In [ ]:
train_ftrs = train_ftrs.na.fill({'year': 'mode'})

In [ ]:
# 1.) deal with `genres` column
# 1.1) lower values...
train_ftrs = train_ftrs.withColumn('genres', F.lower(F.col('genres')))

# 1.2) split value by sep=','
train_ftrs = train_ftrs.withColumn("genres_list", F.split("genres", ",\s*"))

# 1.3) define `is_cartoon` column
train_ftrs = train_ftrs.withColumn('is_cartoon', F.when(F.col('genres').contains('мульт'), 1).otherwise(0))

# 1.4) define `is_adult` column
train_ftrs = train_ftrs.withColumn(
    'is_adult', F.when((F.col('genres').contains('Эро') | F.col('genres').contains('взр')), 1).otherwise(0)
)

# 1.5) define `is_drama` column
train_ftrs = train_ftrs.withColumn(
    'is_drama', F.when((F.col('genres').contains('драм') | F.col('genres').contains('роман')), 1).otherwise(0)
)

# 1.6) define `is_russian` column
train_ftrs = train_ftrs.withColumn(
    'is_russian', 
    F.when(
        (F.col('genres').contains('наши') | F.col('genres').contains('рус') | F.col('genres').contains('совет')), 1
    ).otherwise(0)
)

# 2) define 'is_after_2000' column of content year
train_ftrs = train_ftrs.withColumn('year', F.col('year').cast(T.IntegerType()))
train_ftrs = train_ftrs.withColumn('is_after_2000', F.when(F.col('year') >= 2000, 1).otherwise(0))

In [ ]:
# 3) fraction of purchased content
# 3.1) groupBy
frac_purchased = train_ftrs.groupBy('item_id').agg(
    F.expr('count(item_id)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

# 3.2) get fraction
frac_purchased = frac_purchased.withColumn('frac_purchased_item', F.col('n_purchased') / F.col('count'))
train_new_ftrs = train_ftrs.join(other=frac_purchased.select('item_id', 'frac_purchased_item'), on='item_id')

In [ ]:
# 4) target encoding for 'is_cartoon', 'is_russian', 'is_after_2000'
# 4.1) `is_cartoon`
column = 'is_cartoon'
mean_purchased_cartoon = train_new_ftrs.groupBy(column).agg(
    F.expr('count(is_cartoon)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

mean_purchased_cartoon = mean_purchased_cartoon.withColumn(f'frac_purchased_{column}', F.col('n_purchased') / F.col('count'))
train_new_ftrs = train_new_ftrs.join(
    other=mean_purchased_cartoon.select('is_cartoon', f'frac_purchased_{column}'), on='is_cartoon'
)

# 4.2) `is_russian`
column = 'is_russian'
mean_purchased_cartoon = train_new_ftrs.groupBy(column).agg(
    F.expr('count(is_russian)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

mean_purchased_cartoon = mean_purchased_cartoon.withColumn(f'frac_purchased_{column}', F.col('n_purchased') / F.col('count'))
train_new_ftrs = train_new_ftrs.join(other=mean_purchased_cartoon.select(
    'is_russian', f'frac_purchased_{column}'), on='is_russian'
)

# 4.3) `is_after_2000`
column = 'is_after_2000'
mean_purchased_cartoon = train_new_ftrs.groupBy(column).agg(
    F.expr('count(is_after_2000)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

mean_purchased_cartoon = mean_purchased_cartoon.withColumn(f'frac_purchased_{column}', F.col('n_purchased') / F.col('count'))
train_new_ftrs = train_new_ftrs.join(other=mean_purchased_cartoon.select(
    'is_after_2000', f'frac_purchased_{column}'), on='is_after_2000'
)

# 4.4) `is_adult`
column = 'is_adult'
mean_purchased_cartoon = train_new_ftrs.groupBy(column).agg(
    F.expr('count(is_adult)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

mean_purchased_cartoon = mean_purchased_cartoon.withColumn(f'frac_purchased_{column}', F.col('n_purchased') / F.col('count'))
train_new_ftrs = train_new_ftrs.join(other=mean_purchased_cartoon.select(
    'is_adult', f'frac_purchased_{column}'), on='is_adult'
)

# 4.5) `is_drama`
column = 'is_drama'
mean_purchased_cartoon = train_new_ftrs.groupBy(column).agg(
    F.expr('count(is_drama)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

mean_purchased_cartoon = mean_purchased_cartoon.withColumn(f'frac_purchased_{column}', F.col('n_purchased') / F.col('count'))
train_new_ftrs = train_new_ftrs.join(other=mean_purchased_cartoon.select(
    'is_drama', f'frac_purchased_{column}'), on='is_drama'
)

### laba03_views_programmes.csv

In [ ]:
# 2) deal with 'ts_start' and `ts_end`
# 2.1) convert to datetime format
vws_prgrmms = vws_prgrmms.withColumn('timestamp_start', F.from_unixtime("ts_start").cast("date"))
vws_prgrmms = vws_prgrmms.withColumn('timestamp_end', F.from_unixtime("ts_end").cast("date"))

# 3) create `view_volume_difference` column
vws_prgrmms = vws_prgrmms.withColumn('view_volume_difference_hours', (F.col('ts_end') - F.col('ts_start')) / 60 / 60)

In [ ]:
# 4) make binary variable `item_type`
vws_prgrmms = vws_prgrmms.withColumn('is_live', F.when(F.col('item_type') == 'live', 1).otherwise(0)) 

In [ ]:
# 5) group by `user_id` and get mean features `is_live`, `view_volume_difference_hours`
usr_vws = vws_prgrmms.groupBy('user_id').agg(
    F.sum('view_volume_difference_hours').alias('total_view'),
    F.min('view_volume_difference_hours').alias('min_view'),
    F.max('view_volume_difference_hours').alias('max_view'),
    F.mean('view_volume_difference_hours').alias('avg_view'),
    F.mean('is_live').alias('live_percentage')
)
train_new_ftrs = train_new_ftrs.join(other=usr_vws, on=['user_id'], how='left')

### final result

In [ ]:
drp_clmns = ['genres', 'title']
train_new_ftrs = train_new_ftrs.drop(*drp_clmns)

train_new_ftrs = train_new_ftrs.withColumn('year', F.col('year').cast(T.IntegerType()))
train_new_ftrs = train_new_ftrs.withColumn('purchase', F.col('purchase').cast(T.IntegerType()))
train_new_ftrs = train_new_ftrs.withColumn('item_id', F.col('item_id').cast(T.IntegerType()))
train_new_ftrs = train_new_ftrs.withColumn('content_type', F.col('content_type').cast(T.IntegerType()))

## fit GBTClassifier

In [ ]:
# define fitting features
fttng_ftrs = train_new_ftrs.drop(
    'purchase', 'genres_list', 'user_id', 'item_id', 'is_cartoon', 'is_russian', 'is_after_2000', 'is_adult', 'is_drama'
).columns
# getNaNs(data=train_ftrs.select(*fttng_ftrs))

In [ ]:
# replace NaNs with mean or mode values
train_new_ftrs = train_new_ftrs.na.fill({
    'total_view': 'mean', 'min_view': 'mode', 'max_view': 'mode', 'avg_view': 'mean',
    'live_percentage': 'mean'
})

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier

vctr_assmblr = VectorAssembler(
    inputCols=train_new_ftrs.drop(
        'purchase', 'genres_list', 'user_id', 'item_id', 'is_cartoon', 'is_russian', 'is_after_2000', 'is_adult', 'is_drama'
    ).columns, outputCol="features"
)
train_data = vctr_assmblr.transform(train_new_ftrs.na.drop())

In [ ]:
gbt = GBTClassifier(seed=42, featuresCol='features', labelCol="purchase", maxIter=2)
model = gbt.fit(train_data)
train_prdctns = model.transform(train_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='purchase', metricName='areaUnderROC')
auc = evaluator.evaluate(train_prdctns)
print('AUC: ', auc)

### predict 

In [ ]:
test_ftrs = test_smpl.join(
    other=items_inf.select('item_id', 'content_type', 'title', 'year', 'genres'),
    on=['item_id'], how='left'
)

In [ ]:
test_ftrs = train_ftrs.na.fill({'year': 'mode'})

In [ ]:
# 1.) deal with `genres` column
# 1.1) lower values...
test_ftrs = test_ftrs.withColumn('genres', F.lower(F.col('genres')))

# 1.2) split value by sep=','
test_ftrs = test_ftrs.withColumn("genres_list", F.split("genres", ",\s*"))

# 1.3) define `is_cartoon` column
test_ftrs = test_ftrs.withColumn('is_cartoon', F.when(F.col('genres').contains('мульт'), 1).otherwise(0))

# 1.4) define `is_adult` column
test_ftrs = test_ftrs.withColumn(
    'is_adult', F.when((F.col('genres').contains('Эро') | F.col('genres').contains('взр')), 1).otherwise(0)
)

# 1.5) define `is_drama` column
test_ftrs = test_ftrs.withColumn(
    'is_drama', F.when((F.col('genres').contains('драм') | F.col('genres').contains('роман')), 1).otherwise(0)
)

# 1.6) define `is_russian` column
test_ftrs = test_ftrs.withColumn(
    'is_russian', 
    F.when(
        (F.col('genres').contains('наши') | F.col('genres').contains('рус') | F.col('genres').contains('совет')), 1
    ).otherwise(0)
)

# 2) define 'is_after_2000' column of content year
test_ftrs = test_ftrs.withColumn('year', F.col('year').cast(T.IntegerType()))
test_ftrs = test_ftrs.withColumn('is_after_2000', F.when(F.col('year') >= 2000, 1).otherwise(0))

In [ ]:
# 3) fraction of purchased content
# 3.1) groupBy
frac_purchased = train_ftrs.groupBy('item_id').agg(
    F.expr('count(item_id)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

# 3.2) get fraction
frac_purchased = frac_purchased.withColumn('frac_purchased_item', F.col('n_purchased') / F.col('count'))
test_ftrs = test_ftrs.join(other=frac_purchased.select('item_id', 'frac_purchased_item'), on='item_id')

In [ ]:
# 4) target encoding for 'is_cartoon', 'is_russian', 'is_after_2000'
# 4.1) `is_cartoon`
column = 'is_cartoon'
mean_purchased_cartoon = train_new_ftrs.groupBy(column).agg(
    F.expr('count(is_cartoon)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

mean_purchased_cartoon = mean_purchased_cartoon.withColumn(f'frac_purchased_{column}', F.col('n_purchased') / F.col('count'))
test_ftrs = test_ftrs.join(
    other=mean_purchased_cartoon.select('is_cartoon', f'frac_purchased_{column}'), on='is_cartoon'
)

# 4.2) `is_russian`
column = 'is_russian'
mean_purchased_cartoon = train_new_ftrs.groupBy(column).agg(
    F.expr('count(is_russian)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

mean_purchased_cartoon = mean_purchased_cartoon.withColumn(f'frac_purchased_{column}', F.col('n_purchased') / F.col('count'))
test_ftrs = test_ftrs.join(other=mean_purchased_cartoon.select(
    'is_russian', f'frac_purchased_{column}'), on='is_russian'
)

# 4.3) `is_after_2000`
column = 'is_after_2000'
mean_purchased_cartoon = train_new_ftrs.groupBy(column).agg(
    F.expr('count(is_after_2000)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

mean_purchased_cartoon = mean_purchased_cartoon.withColumn(f'frac_purchased_{column}', F.col('n_purchased') / F.col('count'))
test_ftrs = test_ftrs.join(other=mean_purchased_cartoon.select(
    'is_after_2000', f'frac_purchased_{column}'), on='is_after_2000'
)

# 4.4) `is_adult`
column = 'is_adult'
mean_purchased_cartoon = train_new_ftrs.groupBy(column).agg(
    F.expr('count(is_adult)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

mean_purchased_cartoon = mean_purchased_cartoon.withColumn(f'frac_purchased_{column}', F.col('n_purchased') / F.col('count'))
test_ftrs = test_ftrs.join(other=mean_purchased_cartoon.select(
    'is_adult', f'frac_purchased_{column}'), on='is_adult'
)

# 4.5) `is_drama`
column = 'is_drama'
mean_purchased_cartoon = train_new_ftrs.groupBy(column).agg(
    F.expr('count(is_drama)').alias('count'),
    F.expr('sum(purchase)').alias('n_purchased'),
)

mean_purchased_cartoon = mean_purchased_cartoon.withColumn(f'frac_purchased_{column}', F.col('n_purchased') / F.col('count'))
test_ftrs = test_ftrs.join(other=mean_purchased_cartoon.select(
    'is_drama', f'frac_purchased_{column}'), on='is_drama'
)

In [ ]:
test_ftrs = test_ftrs.join(other=usr_vws, on=['user_id'], how='left')

In [ ]:
drp_clmns = ['genres', 'title']
test_ftrs = test_ftrs.drop(*drp_clmns)

test_ftrs = test_ftrs.withColumn('year', F.col('year').cast(T.IntegerType()))
test_ftrs = test_ftrs.withColumn('user_id', F.col('user_id').cast(T.IntegerType()))
test_ftrs = test_ftrs.withColumn('item_id', F.col('item_id').cast(T.IntegerType()))
test_ftrs = test_ftrs.withColumn('content_type', F.col('content_type').cast(T.IntegerType()))

In [ ]:
# replace NaNs with mean or mode values
test_ftrs = test_ftrs.na.fill({
    'total_view': 'mean', 'min_view': 'mode', 'max_view': 'mode', 'avg_view': 'mean',
    'live_percentage': 'mean'
})

In [ ]:
vctr_assmblr = VectorAssembler(
    inputCols=[
        'is_drama', 'is_adult', 'is_after_2000', 'is_russian', 'is_cartoon', 'item_id', 'content_type', 'year',
        'frac_purchased_item', 'frac_purchased_is_cartoon', 'frac_purchased_is_russian', 'frac_purchased_is_after_2000',
        'frac_purchased_is_adult', 'frac_purchased_is_drama', 'total_view', 'min_view', 'max_view', 'avg_view',
        'live_percentage',
    ], outputCol="features"
)
test_data = vctr_assmblr.transform(test_ftrs.drop('purchase'))

In [ ]:
test_prdctns = model.transform(test_data)

In [ ]:
test_prdctns = test_prdctns.withColumn('user_id', F.col('user_id').cast(T.IntegerType()))
prbblts = test_prdctns.drop('features').select(
    'user_id', 'item_id', 'probability'
)

firstelement = F.udf(lambda v: float(v[1]), T.FloatType())
prbblts = prbblts.withColumn('buy_probability', firstelement('probability'))

In [ ]:
getNaNs(data=prbblts.drop('probability'))

In [ ]:
prbblts_pnds = prbblts.select(
    'user_id', 'item_id', 'buy_probability'
).orderBy(F.col('user_id').asc(), F.col('item_id').asc()).toPandas()

In [ ]:
prbblts_pnds = prbblts_pnds.rename(columns={'buy_probability': 'purchase'})

In [ ]:
prbblts_pnds.to_csv('./lab03.csv')

# spark context stop

In [ ]:
spark.stop()